In [46]:
import numpy as np
import pandas as pd
import seaborn as sns
data = pd.read_csv('labeledTrainData.txt', sep='\t')
data.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [55]:
import string
import re
import nltk
nltk.download('punkt')

from nltk.stem import PorterStemmer
porter = PorterStemmer()

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from nltk.tokenize import word_tokenize

def my_replace(match):
    match = match.group()
    return re.sub('\s', '', match)
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Get negative meaning
    text = re.sub('[not|no|none]+[\s]?[\w]*', my_replace, text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))  
    return text

def tokenizer(text):
    # split into words
    text = word_tokenize(preprocessor(text))
    # convert to lower case
    text = [w.lower() for w in text]
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in text]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    return [w for w in words if not w in stop_words]
     
def tokenizer_porter(text):
    words = tokenizer(text);
    return [porter.stem(word) for word in words]

[nltk_data] Downloading package punkt to /Users/haotran/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [48]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop_words,
                        preprocessor=preprocessor,
                        tokenizer=tokenizer_porter
                        )

In [49]:
y = data['sentiment']
X = data['review']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [50]:
clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preproc...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [51]:
clf.fit(X_train, y_train) 
predictions = clf.predict(X_test)

In [54]:
twits = [
    "This is really bad",
    "I love this!",
    "not good",
]

preds = clf.predict_proba(twits)

for i in range(len(twits)):
    print(f'{twits[i]} --> Positive, Negative = {preds[i]}')

This is really bad --> Positive, Negative = [0.99676344 0.00323656]
I love this! --> Positive, Negative = [0.00433794 0.99566206]
not good --> Positive, Negative = [0.71840171 0.28159829]
